In [17]:
from keras.models import model_from_json
import joblib
from collections import deque
class Trajectory:
    def __init__(self,modelPath='architecture_plane.json',weightsPath='weights_plane.h5',scalerPath = "scaler_plane.pkl"):
        '''
            modelPath to be like = 'architecture.json'
            weightsPath to be like = 'weights.h5'
        '''
        json_file = open(modelPath, 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        self.model = model_from_json(loaded_model_json)
        self.model.load_weights(weightsPath)
        self._look_back = 5
        self.scaler = joblib.load(scalerPath)
        self.scaler.clip = False 
    def predictValues(self,x,lenPred=1):
        '''
        x is the small sequence
        shape should be (no. of rows,5,3)
        -1,3
        Look back is fixed to be 5 and
        3 is the params in each sequence
        lenPred Should be how many further points you want
        '''
        pred = []
        x = np.array(x)
        x = self.scaler.transform(x)
        x.reshape(-1,5,4)    
        x = deque(x)
        assert(len(x) == self._look_back)
        for i in range(lenPred):
            # print("x before",np.array(x).shape)
            x_prime = self.model.predict(np.array(x).reshape((-1,self._look_back,4)))
            x.popleft()
            x.extend([x_prime])
            # print("X_prime",x_prime.shape)
            # print(x)
            # print("x after",np.array(x).shape)
            pred.append(x_prime)
        pred = np.array(pred)
        pred = pred.reshape((-1,4))
        pred = self.scaler.inverse_transform(pred)
        return pred
    def predict(self,x,lenPred):
      x = deque(x)
      pred = []
      for i in range(lenPred):
        xTemp = self.predictValues(x)
        x.popleft()
        x.extend(xTemp)
        pred.append(xTemp)
      return pred

In [18]:
import numpy as np
traj = Trajectory(modelPath="architecture_plane.json",weightsPath="weights_plane.h5",scalerPath = "scaler_plane.pkl")
x = [[[-13954, -9455,2437, 27],[-13925, -9255,2437, 71],[-14265, -9055,2437, 117],[-14615, -8853,2437, 163],[-14974, -8651,2437, 207]]]
x = np.array(x).reshape(-1,4)
# print(x)
val = traj.predict(x,20)

In [19]:
val

[array([[-19883.775  , -11153.218  ,   2535.125  ,    211.25623]],
       dtype=float32),
 array([[-25063.287  , -11331.385  ,   2624.4707 ,    219.36882]],
       dtype=float32),
 array([[-29781.795  , -11741.343  ,   1843.034  ,    223.56526]],
       dtype=float32),
 array([[-33148.668  , -12200.291  ,   1712.3478 ,    223.94424]],
       dtype=float32),
 array([[-34616.656  , -13809.899  ,   1528.5824 ,    224.46129]],
       dtype=float32),
 array([[-35724.215  , -14242.075  ,   1345.8782 ,    225.02797]],
       dtype=float32),
 array([[-35662.54   , -14386.999  ,   1230.3976 ,    224.96556]],
       dtype=float32),
 array([[-35583.305  , -14267.783  ,   1158.1233 ,    225.32933]],
       dtype=float32),
 array([[-35147.035  , -13959.17   ,   1029.423  ,    226.28105]],
       dtype=float32),
 array([[-34383.215  , -13512.872  ,    956.3523 ,    227.15178]],
       dtype=float32),
 array([[-33528.355 , -12856.098 ,    917.5575,    228.4005]],
       dtype=float32),
 array([[-3248

In [20]:
from math import *
import math

In [21]:
#Let Missile Location Coordinates
traj_x = -16494
traj_y = -7800
traj_z = 200

#Let prediction Location Coordinates
traj1_x =-16387.025
traj1_y = -9707.095
traj1_z = 80.613

#let Actual Cordinates of Air Object
traj_currx = -14974
traj_curry = -8651
traj_currz = 207

#Let Velocity of Target Shooting Missile

initial_velocity = 410

distance_x = traj_x - traj1_x
distance_y = traj_y - traj1_y
distance_z = traj_x - traj1_z

theta = math.atan(distance_x / distance_y)
phi = math.atan(distance_z / distance_x)

#Let Acceleration Due To Gravity is
g = 9.8

#Time needed by Object to move at predicted Position

Time_needed = 5

#if (Time_needed_to reach predicted point == Time Needed by MIssile to hit the object) therfore we can shoot at that point of time.

Distance = sqrt((traj1_x - traj_x)**2 + (traj1_y - traj_y)**2 + (traj1_z - traj_z)**2)
print(Distance) 
print(initial_velocity * math.cos(theta)*math.sin(phi)*Time_needed - (0.5*g*((Time_needed)**2)))
if abs(Distance - (initial_velocity * math.cos(theta)*math.sin(phi)*Time_needed - (0.5*g*((Time_needed)**2)))) <= 15:
    print("We can shoot")
else:
    print("Wait For the Next Interception")
    print(initial_velocity * math.cos(theta)*math.sin(phi)*Time_needed - (0.5*g*((Time_needed)**2)))


1913.8203273606948
1924.2398571596063
We can shoot
